In [1]:
library(locfit)
library(RcppCNPy)
library(reticulate)
np <- import('numpy')

locfit 1.5-9.1 	 2013-03-22


In [2]:
p0 = c(1.33738, 13.98811, 0.55950, 11.95797, 12.82356, 0., 0.)

In [3]:
##ngal(1)+wp(19)+ggl(20)+vpf(20)+Pcic(40)

In [4]:
seedlist <- c(361,561,865,932,948)

In [5]:
data <- c(np$load('Run_062718/bolp_21p0_um0_b_361.npz'),
         np$load('Run_062718/bolp_21p0_um0_b_516.npz'),
         np$load('Run_062718/bolp_21p0_um0_b_865.npz'),
         np$load('Run_062718/bolp_21p0_um0_b_932.npz'),
         np$load('Run_062718/bolp_21p0_um0_b_948.npz'))

In [6]:
setwd('~/Assembly Bias Project/Fisher analysis/Run_062718/')

### GCV for alpha

In [7]:
alphas <- seq(0.1,0.9,0.05)

In [8]:
calc_gcv_score <- function(i, j){
    x <- data[[i]]$f[['param']][(500*j-499):(500*j),j]
    gcv_score <- matrix(0,ncol=17,nrow=100)
    for (k in seq(100)){
        y <- data[[i]]$f[['func_all']][(500*j-499):(500*j),k]
        for (l in seq(17)){
            gcv_score[k,l] <- as.numeric(gcv(y~x,kern='tria',alpha=alphas[l]))[4]
        }
    }
    gcv_score
}

In [9]:
for (i in seq(5)){
    np$savez(sprintf('gcv_bolp_20p0_um0_%i.npz',seedlist[i]),gcv_alpha=calc_gcv_score(i,1),
            gcv_logM1=calc_gcv_score(i,2),gcv_sigmalogM=calc_gcv_score(i,3),gcv_logM0=calc_gcv_score(i,4),
            gcv_logMmin=calc_gcv_score(i,5),gcv_Acen=calc_gcv_score(i,6),gcv_Asat=calc_gcv_score(i,7))
}

### locfit with alpha=0.3,0.5,0.7

In [19]:
setwd('~/Assembly Bias Project/Fisher analysis/Run_062718/bolp_20p0_um0_locfit_test_alpha')

In [12]:
find_dfdp_test <- function(data, num, alpha10, plot=0){
    dfdp = matrix(0,nrow=7,ncol=100)
    for (i in seq(7)){
        x <- data$f[['param']][(500*i-499):(500*i),i]
        p <- p0[i]
        for (j in seq(100)){
            y <- data$f[['func_all']][(500*i-499):(500*i),j]
            fit <- locfit(y~x,deriv=1,alpha=alpha10/10.,kern='tria')
            fit_orig <- locfit(y~x,alpha=alpha10/10.,kern='tria')
            f <- as.numeric(preplot(fit_orig,p)[2])
            fp <- as.numeric(preplot(fit,p)[2])
            dfdp[i,j] <- fp
            if (plot==1){
                png(filename=sprintf('run%i_p%if%i_a%i.png',num,(i-1),(j-1),alpha10))
                plot(x, y)
                lines(fit_orig,col='blue')
                abline(v=p,lty=2)
                abline(a=-p*fp+f,b=fp,col='red')
                dev.off()
            }
        }
    }
    dfdp
}

In [16]:
dfdp_236_3 = find_dfdp_test(data[[1]],seedlist[1],alpha10=3,plot=1)
#dfdp_483_3 = find_dfdp_test(data[[2]],seedlist[2],alpha10=3,plot=1)
#dfdp_518_3 = find_dfdp_test(data[[3]],seedlist[3],alpha10=3,plot=1)
#dfdp_563_3 = find_dfdp_test(data[[4]],seedlist[4],alpha10=3,plot=1)
#dfdp_831_3 = find_dfdp_test(data[[5]],seedlist[5],alpha10=3,plot=1)

In [17]:
dfdp_236_5 = find_dfdp_test(data[[1]],seedlist[1],alpha10=5,plot=1)
#dfdp_483_5 = find_dfdp_test(data[[2]],seedlist[2],alpha10=5,plot=1)
#dfdp_518_5 = find_dfdp_test(data[[3]],seedlist[3],alpha10=5,plot=1)
#dfdp_563_5 = find_dfdp_test(data[[4]],seedlist[4],alpha10=5,plot=1)
#dfdp_831_5 = find_dfdp_test(data[[5]],seedlist[5],alpha10=5,plot=1)

In [18]:
dfdp_236_7 = find_dfdp_test(data[[1]],seedlist[1],alpha10=7,plot=1)
#dfdp_483_7 = find_dfdp_test(data[[2]],seedlist[2],alpha10=7,plot=1)
#dfdp_518_7 = find_dfdp_test(data[[3]],seedlist[3],alpha10=7,plot=1)
#dfdp_563_7 = find_dfdp_test(data[[4]],seedlist[4],alpha10=7,plot=1)
#dfdp_831_7 = find_dfdp_test(data[[5]],seedlist[5],alpha10=7,plot=1)

### Use 0.7(default) for now.

In [7]:
setwd('~/Assembly Bias Project/Fisher analysis/Run_062718/bolp_21p0_um0_locfit_tcub_alpha07/')

In [8]:
find_dfdp <- function(data, num, plot=0){
    dfdp = matrix(0,nrow=7,ncol=100)
    for (i in seq(7)){
        x <- data$f[['param']][(500*i-499):(500*i),i]
        p <- p0[i]
        for (j in seq(100)){
            y <- data$f[['func_all']][(500*i-499):(500*i),j]
            fit <- locfit(y~x,deriv=1)
            fit_orig <- locfit(y~x)
            f <- as.numeric(preplot(fit_orig,p)[2])
            fp <- as.numeric(preplot(fit,p)[2])
            dfdp[i,j] <- fp
            if (plot==1){
                png(filename=sprintf('run%i_p%if%i.png',num,(i-1),(j-1)))
                plot(x, y)
                lines(fit_orig,col='blue')
                abline(v=p,lty=2)
                abline(a=-p*fp+f,b=fp,col='red')
                dev.off()
            }
        }
    }
    dfdp
}

In [9]:
dfdp_361 = find_dfdp(data[[1]],seedlist[1],plot=1)
dfdp_561 = find_dfdp(data[[2]],seedlist[2],plot=1)
dfdp_865 = find_dfdp(data[[3]],seedlist[3],plot=1)
dfdp_932 = find_dfdp(data[[4]],seedlist[4],plot=1)
dfdp_948 = find_dfdp(data[[5]],seedlist[5],plot=1)

In [10]:
setwd('~/Assembly Bias Project/Fisher analysis/Run_062718/')

In [11]:
np$savez('dfdp_bolp_21p0_um0_tcub_alpha07.npz',dfdp_361=dfdp_361,dfdp_561=dfdp_561,dfdp_865=dfdp_865,
         dfdp_932=dfdp_932,dfdp_948=dfdp_948)